In [1]:
import pandas as pd

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# @title Load Configuraiton 
import hbqaconfig
conf = hbqaconfig.setEnv('local')
# for k,v in conf.items(): print (k,":",v)

In [6]:
df_hbqa_auto= pd.read_csv(conf['QAGS_FOLDER']+'06.11-ChatGPTAPI-Response_QA_Final.csv')
print(df_hbqa_auto.shape)
df_hbqa_auto.head(4)

(286, 8)


,Unnamed: 0,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Prompt,QA
0,0,Book01_002,1,7511,1214,1615,"Write 5 questions, corresponding answers from ...",\n\nQ1: Who asked Sauti about his journey?\nA1...
1,1,Book01_002,2,7551,1261,1677,"Write 5 questions, corresponding answers from ...",\n\nQ1: Who asked Sauti about his journey?\nA1...
2,2,Book01_002,3,6986,1162,1545,"Write 5 questions, corresponding answers from ...",\nQ1: Who composed the Bharata?\nA1: The Rishi...
3,3,Book01_002,5,10832,1949,2592,"Write 5 questions, corresponding answers from ...",\n\nQ1. Who asked Sauti about his journey?\nA1...


In [270]:
df_hbqa_auto.drop(columns=['WordsInQues','WordsInAns'],inplace=True)

In [ ]:
df_hbqa_auto.Chunk_Id.unique()

### Preparing 06-HBQA_Manual for Merging with 06-HBQA_Auto

In [319]:
df_hbqa_manual=pd.read_csv(r'D:\github\HBQA\DBA\Data\\06.2-HBQA_Manual.csv')
print(df_hbqa_manual.shape)
df_hbqa_manual.head(4)

(1119, 6)


,QuesId,SectionId,ContextId,Question,Answer,Reference
0,1,Book01_001,1,Who composed the Mahabharata and recited it at...,The Mahabharata was composed by Krishna-Dwaipa...,"Reference: ""Sauti said, 'Having heard the dive..."
1,2,Book01_001,1,What is the nature of the Mahabharata and what...,"The Mahabharata is a sacred work, an eminent n...","Reference: ""The Purana, first promulgated by t..."
2,3,Book01_001,1,How did creation begin according to the Mahabh...,"At the beginning of the Yuga, from a mighty eg...","Reference: ""In this world, when it was destitu..."
3,4,Book01_001,2,How will the universe be affected at the end o...,At the end of the world and the expiration of ...,"Reference: ""And what is seen in the universe, ..."


#### <font color=red> Run just below cell only once</font>

In [320]:
# Renaming columns
df_hbqa_manual.rename(columns= {"QuesId":"Ques_Id","SectionId":"Section_Id","ContextId":"Chunk_Id","Answer":"Ref_Answer"}, inplace=True)

# Drop Column
df_hbqa_manual.drop(['Reference'], axis=1, inplace=True)

In [321]:
df_hbqa_manual.head(3)

,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer
0,1,Book01_001,1,Who composed the Mahabharata and recited it at...,The Mahabharata was composed by Krishna-Dwaipa...
1,2,Book01_001,1,What is the nature of the Mahabharata and what...,"The Mahabharata is a sacred work, an eminent n..."
2,3,Book01_001,1,How did creation begin according to the Mahabh...,"At the beginning of the Yuga, from a mighty eg..."


In [322]:
df_hbqa_manual_prompt=pd.read_csv(datapath+'04-Chunk_with_Prompts_for_Manual.csv')

In [323]:
print(df_hbqa_manual_prompt.shape)
df_hbqa_manual_prompt.head(4)

(7315, 5)


,SetChunk_Len,Prompt_Len,Section_Id,Chunk_Id,Prompt
0,6000,5985,Book01_001,1,"Write 3 questions, corresponding answers, and ..."
1,6000,5674,Book01_001,2,"Write 3 questions, corresponding answers, and ..."
2,6000,5169,Book01_001,3,"Write 3 questions, corresponding answers, and ..."
3,6000,5803,Book01_001,4,"Write 3 questions, corresponding answers, and ..."


In [324]:
import pandas as pd
import re

# Define a regular expression pattern to match the desired text
pattern = r'Text: (["\']?)(.+?)\1(?=\nQuestions:|\nAnswer:|\nReference:|$)'

# Create a function to extract the desired text
def extract_text(text):
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(2).strip()
    else:
        return text

# Apply the extract_text function to the 'Prompt' column
df_hbqa_manual_prompt['Chunk'] = df_hbqa_manual_prompt['Prompt'].apply(extract_text)

In [325]:
df_hbqa_manual_prompt.head(3)

,SetChunk_Len,Prompt_Len,Section_Id,Chunk_Id,Prompt,Chunk
0,6000,5985,Book01_001,1,"Write 3 questions, corresponding answers, and ...",THE MAHABHARATA ADI PARVA Section I\nOm! Havin...
1,6000,5674,Book01_001,2,"Write 3 questions, corresponding answers, and ...","And what is seen in the universe, whether anim..."
2,6000,5169,Book01_001,3,"Write 3 questions, corresponding answers, and ...","""Ganesa, upon hearing this address, thus answe..."


In [326]:
df_hbqa_manual = df_hbqa_manual.merge(df_hbqa_manual_prompt,how="left",on="Chunk_Id")\
    [['Ques_Id','Section_Id_x', 'Question', 'Ref_Answer','Chunk_Id','Chunk']]


In [327]:
df_hbqa_manual.head(3)

,Ques_Id,Section_Id_x,Question,Ref_Answer,Chunk_Id,Chunk
0,1,Book01_001,Who composed the Mahabharata and recited it at...,The Mahabharata was composed by Krishna-Dwaipa...,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...
1,2,Book01_001,What is the nature of the Mahabharata and what...,"The Mahabharata is a sacred work, an eminent n...",1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...
2,3,Book01_001,How did creation begin according to the Mahabh...,"At the beginning of the Yuga, from a mighty eg...",1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...


#### <font color=red> Run just below cell only once</font>

In [328]:
# df_hbqa_manual['Chunk_Id']=df_hbqa_manual['Chunk_Id']+10000
df_hbqa_manual['Ques_Id']=df_hbqa_manual['Ques_Id']+10000

In [329]:
df_hbqa_manual.rename({"Section_Id_x":"Section_Id"}, axis=1,inplace=True)

In [330]:
df_hbqa_manual.head(3)

,Ques_Id,Section_Id,Question,Ref_Answer,Chunk_Id,Chunk
0,10001,Book01_001,Who composed the Mahabharata and recited it at...,The Mahabharata was composed by Krishna-Dwaipa...,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...
1,10002,Book01_001,What is the nature of the Mahabharata and what...,"The Mahabharata is a sacred work, an eminent n...",1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...
2,10003,Book01_001,How did creation begin according to the Mahabh...,"At the beginning of the Yuga, from a mighty eg...",1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...


In [331]:
df_hbqa_manual.to_csv(datapath+'06.2-HBQA_Manual_with_Chunk.csv')

In [332]:
df_hbqa_manual.drop(columns=['Chunk_Id'], inplace=True)

In [333]:
df_hbqa_manual.head(3)

,Ques_Id,Section_Id,Question,Ref_Answer,Chunk
0,10001,Book01_001,Who composed the Mahabharata and recited it at...,The Mahabharata was composed by Krishna-Dwaipa...,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...
1,10002,Book01_001,What is the nature of the Mahabharata and what...,"The Mahabharata is a sacred work, an eminent n...",THE MAHABHARATA ADI PARVA Section I\nOm! Havin...
2,10003,Book01_001,How did creation begin according to the Mahabh...,"At the beginning of the Yuga, from a mighty eg...",THE MAHABHARATA ADI PARVA Section I\nOm! Havin...


In [ ]:
# df_hbqa_manual['WordsInQues'] = df_hbqa_manual.Question.str.split(' ').apply(len)
# df_hbqa_manual['WordsInAns']  = df_hbqa_manual.Ref_Answer.str.split(' ').apply(len)
# df_hbqa_manual.head(3)

### Merge Auto and Manual Created QA

In [ ]:
df_hbqa_auto.Chunk_Id.unique()

In [279]:
df_hbqa = pd.concat([df_hbqa_manual,df_hbqa_auto])
print(df_hbqa.shape)
df_hbqa.sample(5)

(2206, 5)


,Ques_Id,Section_Id,Question,Ref_Answer,Chunk
987,10988,Book01_203,Whose advice does Duryodhana seek in order to ...,Duryodhana seeks his father's advice to create...,"Duryodhana said. 'Let us, O father, by means o..."
290,290,Book01_070,What is the story of Aranya in this parva?,The story of Aranya in this parva is about Dha...,"Section LXIX\n(Sambhava Parva continued)\n""Jan..."
245,245,Book01_064,What happened to Astika when his time came?,"Astika ascended to heaven when his time came, ...",Section LXIII\n(Adivansavatarana Parva continu...
520,10521,Book01_095,"Who was the wife of Hasti, and who was their son?","Hasti married Yasodhara, the princess of Triga...","""It is for this (i.e., because the king suppor..."
393,393,Book01_095,What did Garuda do to pass through the wheel o...,Garuda diminished his body and passed through ...,"Section XCIV\n(Sambhava Parva continued)\n""Jan..."


In [280]:
df_hbqa.to_csv(datapath+'06.3-HBQA_Combined.csv')

### Check Duplicate Answer

In [281]:
df_hbqa_unique_ans = pd.DataFrame(df_hbqa.groupby('Ref_Answer')['Ques_Id'].count().sort_values(ascending=False)).reset_index()
df_hbqa_unique_ans

,Ref_Answer,Ques_Id
0,The thirteenth Parva is called Anusasana.,6
1,Indra's fault was that he insulted the Valakhi...,5
2,The Rishi insulted by King Parikshit's father ...,5
3,The Parva is called Karna.,5
4,Takshaka asked Kasyapa to revive a banian tree...,5
...,...,...
1711,Kunti asked Vayu to give her a child endowed w...,1
1712,"Krishna-Dwaipayana, also known as Vyasa or Ved...",1
1713,Krishna observed Arjuna contemplating Bhadra w...,1
1714,"Krishna identified Yudhishthira, Bhima, Arjuna...",1


### Check Duplicate Questions

In [282]:
df_hbqa_unique_ques = pd.DataFrame(df_hbqa.groupby('Question')['Ques_Id'].count().sort_values(ascending=False)).reset_index()
df_hbqa_unique_ques


,Question,Ques_Id
0,Who was the father of Astika?,6
1,How did Kasyapa pacify the Valakhilyas?,6
2,What did Narada declare about Krishna and Arjuna?,5
3,Who followed Yudhishthira into the wilderness?,5
4,What is the name of the thirteenth Parva?,5
...,...,...
1665,What did Pandu command Kunti to do?,1
1666,What did Mandapala do with the four sons he be...,1
1667,What did Kunti ask Vayu for?,1
1668,What did Krishna do with the alms they obtaine...,1


In [283]:
df_hbqa_unique_ques.to_csv(datapath+'07.1-HBQA_UniqueQuesOnly.csv')

In [284]:
df_hbqa_unique_ques.loc[ df_hbqa_unique_ques.Ques_Id>1,: ].to_csv(datapath+'07.2-HBQA_DuplicateQuesOnly.csv')

In [285]:
df_hbqa_unique_ques.loc[ df_hbqa_unique_ques.Ques_Id==1,: ].to_csv(datapath+'07.3-HBQA_NonDuplicatedQuesOnly.csv')

#### A Answer Duplicated How Many Time?

In [286]:
check_answer = df_hbqa_unique_ans.loc[2,'Ref_Answer']
df_hbqa.loc[df_hbqa.Ref_Answer==check_answer,:]

,Ques_Id,Section_Id,Question,Ref_Answer,Chunk
216,216,Book01_056,Who was the Rishi insulted by King Parikshit's...,The Rishi insulted by King Parikshit's father ...,"Section LV\n(Astika Parva continued)\n""Astika ..."
447,447,Book01_105,Who was the Rishi insulted by King Parikshit's...,The Rishi insulted by King Parikshit's father ...,"Section CIV\n(Sambhava Parva continued)\n""Bhis..."
722,722,Book02_003,What was the name of the Rishi insulted by Kin...,The Rishi insulted by King Parikshit's father ...,"A1:Section III\n""Vaisampayana said,--""Then May..."
891,891,Book03_100,What was the name of the Rishi insulted by Kin...,The Rishi insulted by King Parikshit's father ...,"A1:Section C\n""Yudhishthira said, 'O best of r..."
1058,1058,Book03_212,What was the name of the Rishi insulted by Kin...,The Rishi insulted by King Parikshit's father ...,"A1:Section CCXII\n""The Brahmana enquired, 'How..."


#### A Question Duplicated How Many Time?

In [287]:
check_question = df_hbqa_unique_ques.loc[0,'Question']
df_hbqa.loc[df_hbqa.Question==check_question,:]

,Ques_Id,Section_Id,Question,Ref_Answer,Chunk
116,116,Book01_024,Who was the father of Astika?,Jaratkaru was the father of Astika.,"Section XXIII\n(Astika Parva continued)\n""Saut..."
347,347,Book01_083,Who was the father of Astika?,Jaratkaru was the father of Astika.,"Section LXXXII\n(Sambhava Parva continued)\n""V..."
546,546,Book01_141,Who was the father of Astika?,"The father of Astika was Jaratkaru, a powerful...","A1:""The king of the Panchalas, beholding his g..."
664,664,Book01_212,Who was the father of Astika?,Jaratkaru was the father of Astika.,A1:Section CCXI\n(Rajya-labha Parva continued)...
833,833,Book03_076,Who was the father of Astika?,Jaratkaru was the father of Astika.,"A1:Section LXXVI\n""Vrihadaswa said, 'Beholding..."
998,998,Book03_191,Who was the father of Astika?,Jaratkaru was the father of Astika.,A1:come and some time after the king begot thr...


### Preparing 04-Chunk_with_Prompts_for_Manual for Merging with 04-Chunk_with_Prompts

In [ ]:
# df_chunk_for_manul = pd.read_csv(datapath+'03-Chunked-Book_for_Manual.csv')
# print(df_chunk_for_manul.shape)
# df_chunk_for_manul.head(3)

In [57]:
# df_chunk_for_manul.drop(['SetChunk_Len'], axis=1, inplace=True)

<font color=red>Run below cell only once</font>

In [58]:
# df_chunk_for_manul['Chunk_Id'] = df_chunk_for_manul['Chunk_Id'] + 10000

In [59]:
# df_chunk_for_manul['Chunk_Words'] = df_chunk_for_manul.Chunk.str.split(' ').apply(len)
# df_chunk_for_manul['Chunk_Approx_Tokens'] = (df_chunk_for_manul['Chunk_Words']/4).round(0)

In [ ]:
# df_chunk_for_manul.sample(4)

### Merge Chunk Dataset

In [ ]:
# df_chunk = pd.read_csv(datapath+'03-Chunked_Book.csv')
# print(df_chunk.shape)
# df_chunk.head(3)

In [62]:
# df_chunk.drop(['Approx_Ques'], axis=1, inplace=True)

In [ ]:
# df_chunk_final = pd.concat([df_chunk,df_chunk_for_manul])
# df_chunk_final.shape

In [ ]:
# df_chunk_final.sample(5)

In [67]:
# df_chunk_final.to_csv(datapath + '03-Chunked-Book-Combined.csv')

### Create HBQA Final Dataset
Get Quest_Id, Section_Id, Chunk_Id, Chunk, Question, Answer of these unique questions.

In [295]:
df_mahabharat = pd.DataFrame(columns=['Ques_Id','Section_Id','Question','Ref_Answer','Chunk'])
df_mahabharat

,Ques_Id,Section_Id,Question,Ref_Answer,Chunk


In [294]:
temp

,Ques_Id,Section_Id,Question,Ref_Answer,Chunk
116,116,Book01_024,Who was the father of Astika?,Jaratkaru was the father of Astika.,"Section XXIII\n(Astika Parva continued)\n""Saut..."


In [296]:
rowid=0
for row in range(df_hbqa_unique_ques.shape[0]):
    ques= df_hbqa_unique_ques.loc[row,'Question']
    #print(df_hbqa.loc[df_hbqa['Question']==ques,:])

    temp = df_hbqa.loc[df_hbqa['Question']==ques,:].head(1)
    
    # chunk_id = temp['Chunk_Id'].values[0]
    section_id = temp['Section_Id'].values[0]
    ques_id = temp['Ques_Id'].values[0]
    ref_answer = temp['Ref_Answer'].values[0]
    chunk = temp['Chunk'].values[0]

    # chunk = df_chunk_final.loc[df_chunk_final.Chunk_Id==chunk_id,'Chunk'].values[0]

    row = (ques_id, section_id, ques,ref_answer,chunk)
    print("Adding ",ques_id)
    df_mahabharat.loc[rowid] = row
    rowid+=1
    
    # break

Adding  116
Adding  152
Adding  263
Adding  30
Adding  70
Adding  41
Adding  172
Adding  197
Adding  81
Adding  151
Adding  74
Adding  32
Adding  549
Adding  0
Adding  78
Adding  539
Adding  537
Adding  578
Adding  538
Adding  1
Adding  212
Adding  492
Adding  520
Adding  493
Adding  629
Adding  732
Adding  606
Adding  725
Adding  13
Adding  691
Adding  747
Adding  728
Adding  749
Adding  726
Adding  211
Adding  727
Adding  712
Adding  665
Adding  666
Adding  642
Adding  673
Adding  678
Adding  192
Adding  632
Adding  190
Adding  595
Adding  729
Adding  593
Adding  647
Adding  635
Adding  611
Adding  748
Adding  631
Adding  652
Adding  716
Adding  592
Adding  12
Adding  618
Adding  707
Adding  626
Adding  661
Adding  174
Adding  690
Adding  704
Adding  702
Adding  10
Adding  740
Adding  627
Adding  735
Adding  710
Adding  643
Adding  605
Adding  705
Adding  668
Adding  682
Adding  638
Adding  662
Adding  746
Adding  657
Adding  660
Adding  722
Adding  654
Adding  200
Adding  628
Adding

In [297]:
df_mahabharat

,Ques_Id,Section_Id,Question,Ref_Answer,Chunk
0,116,Book01_024,Who was the father of Astika?,Jaratkaru was the father of Astika.,"Section XXIII\n(Astika Parva continued)\n""Saut..."
1,152,Book01_035,How did Kasyapa pacify the Valakhilyas?,Kasyapa pacified the Valakhilyas by telling th...,Section XXXIV\n(Astika Parva continued)\n'Saut...
2,263,Book01_066,What did Narada declare about Krishna and Arjuna?,Narada declared that Krishna and Arjuna were N...,"Section LXV\n(Sambhava Parva)\n""Vaisampayana s..."
3,30,Book01_003,Who followed Yudhishthira into the wilderness?,Snatakas and noble-minded Brahmanas who live u...,"'Listen, O ye ascetics, to the outlines of the..."
4,70,Book01_004,What is the name of the thirteenth Parva?,The thirteenth Parva is called Anusasana.,"""In the beginning, ye created the ten points o..."
...,...,...,...,...,...
1665,10667,Book01_123,What did Pandu command Kunti to do?,Pandu commanded Kunti to ask for an offspring ...,"""Pandu, having obtained that virtuous son, aga..."
1666,11101,Book01_231,What did Mandapala do with the four sons he be...,Mandapala left his four sons with their mother...,"""The celestials answered, 'Hear, O Brahmana, o..."
1667,10669,Book01_123,What did Kunti ask Vayu for?,Kunti asked Vayu to give her a child endowed w...,"""Pandu, having obtained that virtuous son, aga..."
1668,10956,Book01_195,What did Krishna do with the alms they obtaine...,Krishna devoted a portion of the alms to the g...,"""Vaisampayana said, 'Thus addressed Dhrishtady..."


In [298]:
# Finally save 
df_mahabharat.to_csv(datapath+'07.4-MahabharatQA.csv')